# Basic D3 Visualization

Step 1: Make some chartable data

In [1]:
import gzip
import numpy as np

In [25]:
def load_wvec(fname, vocab=10000):                                                                                                                                                                                      
    """                                                                                                                                                                                                              
    Loads 300x1 word vecs from Google (Mikolov) word2vec                                                                                                                                                             
    """                                                                                                                                                                                                              
    word_vecs = {}                                                                                                                                                                                                   
    with open(fname, "rb") as f:                                                                                                                                                                                     
        header = f.readline()                                                                                                                                                                                        
        vocab_size, layer1_size = map(int, header.split())                                                                                                                                                           
        binary_len = np.dtype('float32').itemsize * layer1_size
        
        vecs = np.zeros((vocab, layer1_size))
        print(vecs.shape)
        words = []
        for i, line in enumerate(range(vocab)):
            word = []
            while True:                                                                                                                                                                                              
                ch = f.read(1)
                if ch == b' ':
                    word = ''.join(word)
                    break
                if ch != b'\n':
                    word.append(ch.decode("utf-8", "ignore"))                
            vecs[i] = np.fromstring(f.read(binary_len), dtype='float32')                                                                                                                                  
            words.append(word)
# #             else:                                                                                                                                                                                                    
# #                 f.read(binary_len)                                                                                                                                                                                   
    return (vecs, words)
word_vecs, words  = load_wvec("/home/srush/data/GoogleNews-vectors-negative300.bin", 1000)

(1000, 300)


In [26]:
import sklearn.decomposition
import sklearn.manifold

pca = sklearn.decomposition.pca.PCA(n_components=2).fit_transform(word_vecs)
tsne = sklearn.manifold.t_sne.TSNE(n_components=2)\
    .fit_transform(sklearn.decomposition.pca.PCA(n_components=5).fit_transform(word_vecs))

In [27]:
#import pandas
#df = pandas.DataFrame(word_vecs, words)
#a = df.to_json(orient="records")
#a[:10000]

d = []
for w, vec, vec2 in zip(words, pca, tsne):
    d.append({"word":w, "vals": list(vec), "tsne": list(vec2)})
import json
out = json.dumps(d)

So here's where we actually translate the data to the frontend. I also converted the data on the frontend; you 
could do it in Python if you so desired.

In [34]:
from IPython.display import Javascript
Javascript("""
           window.data_vecs={};
           """.format(out))

<IPython.core.display.Javascript object>

In [29]:
%%javascript
require.config({
    paths: {
        d3: '//cdnjs.cloudflare.com/ajax/libs/d3/4.2.1/d3.min',
        lodash: '//cdnjs.cloudflare.com/ajax/libs/lodash.js/4.14.1/lodash.min'
    }
});

<IPython.core.display.Javascript object>

And here it is! Pretty cool, right? Notably, this chart is interactive and responds to user input (tooltip).

D3 presented below without comment. The point is your arbitrary visualization code will work, too!

In [135]:
%%javascript
window.plot_points = function plot_points(d3, svg, show, zoom, offset){
//     var pieData = [12,31]
//     var colours = ['#F00','#000','#000','#000','#000','#000','#000','#000','#000'];
//     var arc = d3.svg.arc()
//     .outerRadius(100/2 - 10)
//     .innerRadius(0);
    if (show == "pca") {
        showfn = function(d) {return d.vals}
    }
    else {
        showfn = function(d) {return d.tsne}
    }

    var xex = d3.extent(data_vecs, function (d) {return showfn(d)[0];});
    //var ymin = d3.min(data_vecs.map(function(d) {showfn(d)[1]}))
    var yex = d3.extent(data_vecs, function (d) {return showfn(d)[1];});

    
    console.log(xex)
    var x = d3.scaleLinear().domain(xex).range([-(width - offset)/(zoom*2), (width+offset)/(zoom*2)]);
    var y = d3.scaleLinear().domain(yex).range([-height/(zoom*2), height/(zoom*2)]);
    var showfn
    

    
    var dots = svg.selectAll('.dots')
        .data(data_vecs, function(d){return d.word})

    dots.exit().remove();

    dots
       .transition()
        .attr("x", function(d) { return x(showfn(d)[0]); })
        .attr("y", function(d) { return y(showfn(d)[1]); });


    dots
        .enter()        
        .append("text")
        .attr("class", "dots")
        //.attr("r", 3.5)
        .text(function(d) { return d.word; })
        .attr("x", function(d) { return x(showfn(d)[0]); })
        .attr("y", function(d) { return y(showfn(d)[1]); });
        //.attr('stroke','#ffffff');
//        .append("svg:title")

    ;
}

<IPython.core.display.Javascript object>

In [140]:
%%javascript 
window.width = 1000
window.height = 1000
require(['d3'], function(d3){
    $("#chart1").remove();
    $("#bod").remove();
    element.append("<div id='bod'></div>")
    element.append("<svg id='chart1' height="+height+" width="+width+"></svg>");  
    
    var svg = d3.select("#chart1")
                .append('g')
                .attr('transform','translate(' + width/2 + ',' + height/2 + ')');

    plot_points(d3, svg, "pca",1, 0);

    function make_button(t, f) {
        return d3.select('#bod').append("div").append('a')
            .text(t)
            .on('click', f);
    }

    make_button("sne", function(){return plot_points(d3, svg, "t-sne", 1, 0);})
    make_button("pca", function(){return plot_points(d3, svg, "pca", 1, 0);})

    var zoom = 1
    var offset = 0
    make_button('zoom_in', function(){zoom = zoom * 0.5; return plot_points(d3, svg, "t-sne", zoom, offset);})
    make_button('zoom_out', function(){zoom = zoom * 2; return plot_points(d3, svg, "t-sne", zoom, offset);})
    make_button('left', function(){offset = offset - 100; return plot_points(d3, svg, "t-sne", zoom, offset);})
    make_button('right', function(){offset = offset + 100; return plot_points(d3, svg, "t-sne", zoom, offset);})

})

<IPython.core.display.Javascript object>

In [26]:
%%javascript
require(['d3'], function(d3){
  //a weird idempotency thing
  $("#chart1").remove();
  //create canvas
  element.append("<div id='chart1'></div>");
  $("#chart1").width("960px");
  $("#chart1").height("600px");        
  var margin = {top: 20, right: 20, bottom: 30, left: 40};
  var width = 880 - margin.left - margin.right;
  var height = 500 - margin.top - margin.bottom;
  var svg = d3.select("#chart1").append("svg")
    .style("position", "relative")
    .style("max-width", "960px")
    .attr("width", width + "px")
    .attr("height", (height + 50) + "px")
    .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");
  
  //set data
  var data = convertPlotData(window.headwayVsRidership);
  var xVals = function(d){return d.purpleHeadway;};
  var yVals = function(d){return d.purpleAverage;};
  
  var xScale = d3.scale.linear().range([0, width]);
  var xAxis = d3.svg.axis().scale(xScale).orient("bottom");
  var xMap = function(d) { return xScale(xVals(d));};
      
  var yScale = d3.scale.linear().range([height, 0]);
  var yAxis = d3.svg.axis().scale(yScale).orient("left");
  var yMap = function(d) { return yScale(yVals(d));};
  xScale.domain([d3.min(data, xVals)-1, d3.max(data, xVals)+1]);
  yScale.domain([d3.min(data, yVals)-1, d3.max(data, yVals)+1]);

  
  var cValue = function(d) { 
    if(d.day == "Saturday" || d.day == "Sunday"){
        return "Weekend";
    }
    return "Weekday";
  }
  var color = d3.scale.category10(); 
  
  // x-axis
  svg.append("g")
      .attr("class", "x axis")
      .attr("transform", "translate(0," + height + ")")
      .call(xAxis)
    .append("text")
      .attr("class", "label")
      .attr("x", width - 80)
      .attr("y", -15)
      .style("text-anchor", "end")
      .text("Average Headway");

  // y-axis
  svg.append("g")
      .attr("class", "y axis")
      .call(yAxis)
    .append("text")
      .attr("class", "label")
      .attr("transform", "rotate(-90)")
      .attr("y", 0)
      .attr("dy", "1em")
      .style("text-anchor", "end")
      .text("Average Riders");

  //NEW: TOOLTIP. 
  var tooltip = d3.select("body").append("div")
    .attr("class", "tooltip")
    .style("opacity", 0)
    .style("background-color", "white");
 
  svg.selectAll(".dot")
     .data(data)
     .enter().append("circle")
     .attr("class", "dot")
     .attr("r", 3.5)
     .attr("cx", xMap)
     .attr("cy", yMap)
     .style("fill", function(d) { return color(cValue(d));}) //D3 does the magic! 
     .on("mouseover", function(d) { //much like jquery, an event listener
         tooltip.transition()
                .duration(200)
                .style("opacity", .9);
         tooltip.html(d["day"] + " : " + d['date'])
               .style("left", (d3.event.pageX + 5) + "px")
               .style("top", (d3.event.pageY - 28) + "px");
      })
      .on("mouseout", function(d) {
          tooltip.transition()
               .duration(500)
               .style("opacity", 0);
      });
    
  var legend = svg.selectAll(".legend")
      .data(color.domain()) //stores the color <-> label mappings
      .enter().append("g")
      .attr("class", "legend")
      .attr("transform", function(d, i) { return "translate(0," + i * 20 + ")"; });

  legend.append("rect")
      .attr("x", width - 60)
      .attr("width", 18)
      .attr("height", 18)
      .style("fill", color);

  legend.append("text")
      .attr("x", width - 70)
      .attr("y", 9)
      .attr("dy", ".35em")
      .style("text-anchor", "end")
      .text(function(d) { return d;})

});

<IPython.core.display.Javascript object>

In [8]:
%%javascript
require(['d3'], function(d3){
  //a weird idempotency thing
  $("#chart1").remove();
  //create canvas
  element.append("<div id='chart1'></div>");
  $("#chart1").width("960px");
  $("#chart1").height("600px");        
  var margin = {top: 20, right: 20, bottom: 30, left: 40};
  var width = 880 - margin.left - margin.right;
  var height = 500 - margin.top - margin.bottom;
  var svg = d3.select("#chart1").append("svg")
    .style("position", "relative")
    .style("max-width", "960px")
    .attr("width", width + "px")
    .attr("height", (height + 50) + "px")
    .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");
  
  //set data
  var data = convertPlotData(window.headwayVsRidership);
  var xVals = function(d){return d.purpleHeadway;};
  var yVals = function(d){return d.purpleAverage;};
  
  var xScale = d3.scale.linear().range([0, width]);
  var xAxis = d3.svg.axis().scale(xScale).orient("bottom");
  var xMap = function(d) { return xScale(xVals(d));};
      
  var yScale = d3.scale.linear().range([height, 0]);
  var yAxis = d3.svg.axis().scale(yScale).orient("left");
  var yMap = function(d) { return yScale(yVals(d));};
  xScale.domain([d3.min(data, xVals)-1, d3.max(data, xVals)+1]);
  yScale.domain([d3.min(data, yVals)-1, d3.max(data, yVals)+1]);

  
  var cValue = function(d) { 
    if(d.day == "Saturday" || d.day == "Sunday"){
        return "Weekend";
    }
    return "Weekday";
  }
  var color = d3.scale.category10(); 
  
  // x-axis
  svg.append("g")
      .attr("class", "x axis")
      .attr("transform", "translate(0," + height + ")")
      .call(xAxis)
    .append("text")
      .attr("class", "label")
      .attr("x", width - 80)
      .attr("y", -15)
      .style("text-anchor", "end")
      .text("Average Headway");

  // y-axis
  svg.append("g")
      .attr("class", "y axis")
      .call(yAxis)
    .append("text")
      .attr("class", "label")
      .attr("transform", "rotate(-90)")
      .attr("y", 0)
      .attr("dy", "1em")
      .style("text-anchor", "end")
      .text("Average Riders");

  //NEW: TOOLTIP. 
  var tooltip = d3.select("body").append("div")
    .attr("class", "tooltip")
    .style("opacity", 0)
    .style("background-color", "white");
 
  svg.selectAll(".dot")
     .data(data)
     .enter().append("circle")
     .attr("class", "dot")
     .attr("r", 3.5)
     .attr("cx", xMap)
     .attr("cy", yMap)
     .style("fill", function(d) { return color(cValue(d));}) //D3 does the magic! 
     .on("mouseover", function(d) { //much like jquery, an event listener
         tooltip.transition()
                .duration(200)
                .style("opacity", .9);
         tooltip.html(d["day"] + " : " + d['date'])
               .style("left", (d3.event.pageX + 5) + "px")
               .style("top", (d3.event.pageY - 28) + "px");
      })
      .on("mouseout", function(d) {
          tooltip.transition()
               .duration(500)
               .style("opacity", 0);
      });
    
  var legend = svg.selectAll(".legend")
      .data(color.domain()) //stores the color <-> label mappings
      .enter().append("g")
      .attr("class", "legend")
      .attr("transform", function(d, i) { return "translate(0," + i * 20 + ")"; });

  legend.append("rect")
      .attr("x", width - 60)
      .attr("width", 18)
      .attr("height", 18)
      .style("fill", color);

  legend.append("text")
      .attr("x", width - 70)
      .attr("y", 9)
      .attr("dy", ".35em")
      .style("text-anchor", "end")
      .text(function(d) { return d;})

});

<IPython.core.display.Javascript object>